# Phase 1

In [ ]:
import os
import time
from openai import OpenAI
import json

In [ ]:
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
phase1_assistant_id = None
phase1_thread_id = None

In [ ]:
instruction = """
    #Objective:
    As an intelligent customer service chatbot of the “Flora” focus app, my goal is to understand and help you solve problems encountered when using the Flora app effectively.
    #Main features of the Flora app:
    Flora is a new way to stay off your phone, clear to-do lists, and build positive, life-changing habits. Whenever you want to make progress towards your goals, plant a seed in Flora. As you work hard, the seed will grow into a healthy tree. Let the tree be your coach and grow yourself with it. You will be amazed by how great you can be.
    FOCUS TREES
    Have trouble putting down the phone? Flora blocks distracting apps in a pleasant way to help you focus on what’s more important in real life. If you cannot resist the temptation of using your phone and leave the app while growing a tree, the tree will be killed! But if succeeding, you'll unlock new trees.
    DOUBLE EFFICIENCY WITH FRIENDS
    Things get even more interesting when you challenge your friends to plant trees together—you will see who kills a tree (ouch). However, if you successfully stay away from your phones together, each of you will win an additional tree from a random friend. You can also chat and share your progress to motivate each other and keep everyone on track. Flora is the original app for multi-user tree planting and delivers the best features that help you and your friends be productive.
    TO-DO LIST & WIDGET
    Flora is a to-do list, and more. Simply create a to-do item and set a reminder, then you will never forget a thing. Even better, you can tag your trees with a to-do item to easily track the progress towards each of your life goals. No matter if your goal is daily, weekly, or monthly, Flora will help you follow it through and achieve it.
    HABIT TRACKER
    Enjoy a rewarding daily habit routine. If you grow trees regularly, you will earn yourself a beautiful, vibrant garden. Take a look at it and feel proud of yourself! Each tree records your perseverance and a step towards a healthy life. You can also view the daily, weekly, or monthly statistics of your activities in Flora to find opportunities to improve your time-management and planning skills.
    REAL TREES, REAL IMPACT
    Why not plant REAL trees while improving yourself? Flora proudly offers the opt-in Price and Care services that let you plant real trees on the earth if you kill or successfully grow a tree in the app. We partner with tree-planting organizations in Africa and East Asia, such as the Trees.org and Eden Reforestation Projects, to plant fast-growing soil rebuilding trees, fruit trees that diversify incomes and nutrition, and trees that can provide forage and fuel-wood. All these trees help make the earth greener and provide long-term financial aid to families who need it most.
    #Job:
    Step 1: To ensure I fully understand your problem, I will output follow-up questions if your issue is unclear or if the reason behind your problem is not fully explained. Skip to Step 2 if the problem is clear enough for Flora customer service to solve.
    Step 2: After fully understanding your problem, I will summarize the essential, constructive, concise, and objective parts of your input in your tone into key points. 
    #Rule:
    1. Step 1 can be done multiple times, but one question at a time, and the output must be a JSON object using key 'messages' for our Python function to work as a parameter.
    2. Step 2 can only be done once, so summarize key points with caution. The output must be a JSON object using key 'keypoints' for our Python function to work as a parameter.
    3. Ensure each key point includes both the specific issue and the context needed to address it, so each key point can be solved independently.
    4. Consolidate key points from Step 2 that address the same issue into a single, comprehensive key point. Ensure that each consolidated key point includes all relevant details and aspects of the issue to avoid redundancy and provide a clear, unified description of the problem.
    5. If multiple key points from Step 2 summarize the same problem, combine them into a single, comprehensive key point to avoid redundancy.
    6. Each key point will be assigned to one human to solve, and they can't see each other. So, Rules 3, 4, and 5 are very important.
    #Example:
    input: How do I change my password?
    output: {"keypoints": ["My problem is I need to change my password."]}
    input: I am just doing my a-levels at the moment and I love using your app for revision. However, when I went onto the app earlier it logged me out and all my data had been lost. I was just wondering if you could help me out.
    output: {"keypoints": ["My data was lost after being logged out of the app."]}
    input: My flora friend page won't load. If I delete the app and re-download it and then login. Will all my data and garden time be lost?
    output: {"keypoints": ["My Flora friend page will not load.", "I am concerned about losing data and garden time after deleting and reinstalling the app."]}
    input: The app is full of bugs.
    output: {"messages": ["I'm sorry, I didn't understand that. Could you please provide more details or specify the issue you are facing with the Flora app?"]}
"""

In [ ]:
def phase1(input):
    assistant = client.beta.assistants.create(
        name = "Intelligent customer service chatbot",
        instructions = instruction,
        # model = "gpt-3.5-turbo-1106"
        model = "gpt-4o-mini"
    )
    global phase1_assistant_id
    phase1_assistant_id = assistant.id

    thread = client.beta.threads.create()
    global phase1_thread_id
    phase1_thread_id = thread.id

    message = client.beta.threads.messages.create(
        thread_id = thread.id,
        role = "user",
        content = input
    )

    run = client.beta.threads.runs.create(
        thread_id = thread.id,
        assistant_id = assistant.id
    )
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id = thread.id,
            run_id = run.id
        )

    messages = client.beta.threads.messages.list(
        thread_id = thread.id,
        order = 'asc'
    )

    user_or_assistant = 0
    for i in messages.data:
        if user_or_assistant == 0:
            print("User: ")
            user_or_assistant = 1
        elif user_or_assistant == 1:
            print("Assistant: ")
            user_or_assistant = 0
        print(i.content[0].text.value)
    
    # if app need return the last sentence
    # response = messages.data[-1].content[0].text.value
    # return response

In [ ]:
def chat(input):
    message = client.beta.threads.messages.create(
        thread_id = phase1_thread_id,
        role = "user",
        content = input
    )

    run = client.beta.threads.runs.create(
        assistant_id = phase1_assistant_id,
        thread_id = phase1_thread_id
    )
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id = phase1_thread_id,
            run_id = run.id
        )


    # need to handle request erroe
    
    
    # get result message
    messages = client.beta.threads.messages.list(
        thread_id = phase1_thread_id,
        order = 'asc'
    )

    user_or_assistant = 0
    for i in messages.data:
        if user_or_assistant == 0:
            print("User: ")
            user_or_assistant = 1
        elif user_or_assistant == 1:
            print("Assistant: ")
            user_or_assistant = 0
        print(i.content[0].text.value)
        
    # response = messages.data[-1].content[0].text.value
    # return response

# Test

In [ ]:
input = ""
phase1(input)

In [ ]:
input = ""
chat(input)

# JSON Format

In [ ]:
messages = client.beta.threads.messages.list(
    thread_id = phase1_thread_id,
    order = 'asc'
)

conversation_log = {"Past Converstion Log": []}

user_or_assistant = 0
for i in messages.data:
    role = "user" if user_or_assistant == 0 else "assistant"
    message = i.content[0].text.value.replace("\n", " ")

    conversation_log["Past Converstion Log"].append({
        "role": role,
        "message": message
    })
    
    user_or_assistant = 1 - user_or_assistant

conversation_log_json = json.dumps(conversation_log, ensure_ascii=False, indent=4)

keypoints_list = []

try: 
    last_message = conversation_log["Past Converstion Log"][-1]["message"]
    keypoints = json.loads(last_message)["keypoints"]
    for keypoint in keypoints:
        keypoints_list.append(keypoint)
except json.JSONDecodeError as e:
    pass

In [ ]:
print(conversation_log_json)

# Result

In [ ]:
conversation_log_json

In [ ]:
keypoints_list

# Delete Assistant

In [ ]:
client.beta.assistants.delete(phase1_assistant_id)